In [27]:
import requests
from bs4 import BeautifulSoup
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint
import pandas as pd

In [26]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '82b987df-3987-4366-8015-d452dc2f66a9'
}


try:
  response = requests.get(url, headers = headers, params=parameters)
  data = json.loads(response.text)
  print('success')
  pprint.pprint(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)
  print('error')

success
{'data': [{'circulating_supply': 19562843,
           'cmc_rank': 1,
           'date_added': '2010-07-13T00:00:00.000Z',
           'id': 1,
           'infinite_supply': False,
           'last_updated': '2023-12-06T21:02:00.000Z',
           'max_supply': 21000000,
           'name': 'Bitcoin',
           'num_market_pairs': 10599,
           'platform': None,
           'quote': {'USD': {'fully_diluted_market_cap': 919846721370.4,
                             'last_updated': '2023-12-06T21:02:00.000Z',
                             'market_cap': 856896047344.4724,
                             'market_cap_dominance': 53.729,
                             'percent_change_1h': -0.4641775,
                             'percent_change_24h': 0.11821573,
                             'percent_change_30d': 25.07563958,
                             'percent_change_60d': 56.79221081,
                             'percent_change_7d': 16.18341803,
                             'percent_cha

In [28]:
#layers: ['data']['name']      ['data']['quote']['USD']['market_cap']
def get_data(response):
    data = json.loads(response.content)['data']
    rows = list()
    
    for item in data:
        rows.append([
             item['name'], 
             item['quote']['USD']['market_cap'],
             item['quote']['USD']['volume_24h'],
             item['quote']['USD']['last_updated']])

    df = pd.DataFrame(
        rows, columns=['name','market_cap','volume_24h','last_updated'])

    # remove timezone from datetime64[ns, UTC] in `last_updated`
    df['last_updated'] = pd.to_datetime(df.last_updated).dt.tz_localize(None)
    df.index.name = 'id'
    return(df)

In [33]:
df = get_data(response)
df.head()

,name,market_cap,volume_24h,last_updated
id,,,,
0,Bitcoin,8.568960e+11,3.282922e+10,2023-12-06 21:02:00
1,Ethereum,2.713333e+11,1.325357e+10,2023-12-06 21:02:00
2,Tether USDt,8.994263e+10,5.922168e+10,2023-12-06 21:01:00
3,BNB,3.492837e+10,8.304075e+08,2023-12-06 21:01:00
4,XRP,3.418148e+10,1.910788e+09,2023-12-06 21:02:00


In [37]:
df1 = df[['name','market_cap']]
df1.to_csv('latest_market_cap')

The visualization part is implemented in the flourish.